In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!python -m pip install pyyaml
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

fatal: destination path 'detectron2' already exists and is not an empty directory.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment


In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

In [5]:
data_dir_path = "/content/drive/MyDrive/instseg/"

In [6]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("taco10_train", {}, data_dir_path + "mapped_annotations_0_train.json", data_dir_path + "images/")
register_coco_instances("taco10_val", {}, data_dir_path + "mapped_annotations_0_val.json", data_dir_path + "images/")
register_coco_instances("taco10_test", {}, data_dir_path + "mapped_annotations_0_test.json", data_dir_path + "images/")

In [7]:
import json

# Load your JSON file
with open('/content/drive/MyDrive/instseg/mapped_annotations_0_train.json') as f:
    data = json.load(f)

# Generate unique IDs
unique_id = 0
for annotation in data['annotations']:
    annotation['id'] = unique_id
    unique_id += 1

# Save the corrected JSON back to file
with open('/content/drive/MyDrive/instseg/mapped_annotations_0_train.json', 'w') as f:
    json.dump(data, f, indent=4)


In [8]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [9]:
from detectron2.data import build_detection_train_loader
class Trainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    return COCOEvaluator(dataset_name, output_dir=output_folder)

In [10]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("taco10_train",)
cfg.DATASETS.TEST = ("taco10_val",)
cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[08/21 20:34:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[08/21 20:34:02 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[08/21 20:34:18 d2.utils.events]:  eta: 0:04:53  iter: 19  total_loss: 3.114  loss_cls: 2.192  loss_box_reg: 0.4955  loss_mask: 0.08066  loss_rpn_cls: 0.06332  loss_rpn_loc: 0.01225    time: 0.7153  last_time: 0.9706  data_time: 0.3368  last_data_time: 0.7213   lr: 3.8962e-05  max_mem: 2336M
[08/21 20:34:34 d2.utils.events]:  eta: 0:03:51  iter: 39  total_loss: 2.323  loss_cls: 1.431  loss_box_reg: 0.6638  loss_mask: 0.07908  loss_rpn_cls: 0.04767  loss_rpn_loc: 0.01795    time: 0.6803  last_time: 0.8609  data_time: 0.3361  last_data_time: 0.4256   lr: 7.8922e-05  max_mem: 2336M
[08/21 20:34:48 d2.utils.events]:  eta: 0:03:45  iter: 59  total_loss: 1.44  loss_cls: 0.6644  loss_box_reg: 0.487  loss_mask: 0.07694  loss_rpn_cls: 0.04439  loss_rpn_loc: 0.01544    time: 0.6747  last_time: 0.3567  data_time: 0.3536  last_data_time: 0.1184   lr: 0.00011888  max_mem: 2337M
[08/21 20:35:02 d2.utils.events]:  eta: 0:03:31  iter: 79  total_loss: 1.577  loss_cls: 0.6223  loss_box_reg: 0.4998  loss

...

[08/21 20:48:15 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0015 s/iter. Inference: 0.1726 s/iter. Eval: 1.1041 s/iter. Total: 1.2782 s/iter. ETA=0:02:57
[08/21 20:48:20 d2.evaluation.evaluator]: Inference done 17/150. Dataloading: 0.0016 s/iter. Inference: 0.1516 s/iter. Eval: 0.9058 s/iter. Total: 1.0593 s/iter. ETA=0:02:20
[08/21 20:48:25 d2.evaluation.evaluator]: Inference done 22/150. Dataloading: 0.0016 s/iter. Inference: 0.1523 s/iter. Eval: 0.8985 s/iter. Total: 1.0527 s/iter. ETA=0:02:14
[08/21 20:48:30 d2.evaluation.evaluator]: Inference done 23/150. Dataloading: 0.0017 s/iter. Inference: 0.1708 s/iter. Eval: 1.1035 s/iter. Total: 1.2763 s/iter. ETA=0:02:42
[08/21 20:48:37 d2.evaluation.evaluator]: Inference done 24/150. Dataloading: 0.0017 s/iter. Inference: 0.1950 s/iter. Eval: 1.3725 s/iter. Total: 1.5696 s/iter. ETA=0:03:17
[08/21 20:48:43 d2.evaluation.evaluator]: Inference done 29/150. Dataloading: 0.0017 s/iter. Inference: 0.1865 s/iter. Eval: 1.295

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[08/21 20:52:53 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0015 s/iter. Inference: 0.1452 s/iter. Eval: 0.8276 s/iter. Total: 0.9742 s/iter. ETA=0:02:15
[08/21 20:52:58 d2.evaluation.evaluator]: Inference done 19/150. Dataloading: 0.0017 s/iter. Inference: 0.1286 s/iter. Eval: 0.6479 s/iter. Total: 0.7785 s/iter. ETA=0:01:41
[08/21 20:53:06 d2.evaluation.evaluator]: Inference done 23/150. Dataloading: 0.0017 s/iter. Inference: 0.1425 s/iter. Eval: 0.8805 s/iter. Total: 1.0250 s/iter. ETA=0:02:10
[08/21 20:53:12 d2.evaluation.evaluator]: Inference done 26/150. Dataloading: 0.0018 s/iter. Inference: 0.1550 s/iter. Eval: 1.0105 s/iter. Total: 1.1676 s/iter. ETA=0:02:24
[08/21 20:53:19 d2.evaluation.evaluator]: Inference done 31/150. Dataloading: 0.0018 s/iter. Inference: 0.1577 s/iter. Eval: 1.0355 s/iter. Total: 1.1952 s/iter. ETA=0:02:22
[08/21 20:53:24 d2.evaluation.evaluator]: Inference done 34/150. Dataloading: 0.0018 s/iter. Inference: 0.1629 s/iter. Eval: 1.085

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[08/21 20:56:55 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0015 s/iter. Inference: 0.1315 s/iter. Eval: 0.6841 s/iter. Total: 0.8170 s/iter. ETA=0:01:53
[08/21 20:57:01 d2.evaluation.evaluator]: Inference done 22/150. Dataloading: 0.0017 s/iter. Inference: 0.1135 s/iter. Eval: 0.5087 s/iter. Total: 0.6240 s/iter. ETA=0:01:19
[08/21 20:57:08 d2.evaluation.evaluator]: Inference done 24/150. Dataloading: 0.0017 s/iter. Inference: 0.1392 s/iter. Eval: 0.7925 s/iter. Total: 0.9337 s/iter. ETA=0:01:57
[08/21 20:57:14 d2.evaluation.evaluator]: Inference done 31/150. Dataloading: 0.0017 s/iter. Inference: 0.1390 s/iter. Eval: 0.7929 s/iter. Total: 0.9338 s/iter. ETA=0:01:51
[08/21 20:57:20 d2.evaluation.evaluator]: Inference done 36/150. Dataloading: 0.0017 s/iter. Inference: 0.1404 s/iter. Eval: 0.8100 s/iter. Total: 0.9524 s/iter. ETA=0:01:48
[08/21 20:57:25 d2.evaluation.evaluator]: Inference done 39/150. Dataloading: 0.0018 s/iter. Inference: 0.1475 s/iter. Eval: 0.890